## Laden und durchsuchen von eigenen Daten

1. Zuerst laden wir die Dokumente aus dem docs Verzeichniss
2. Dann teilen wir die Dokumente in kleine Segmente auf
3. Dann erstellen wir daraus einen Index

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./docs', glob="**/*.md")
data = loader.load()
len(data)

In [ ]:
print(f"Zeichen in Text {len(data[1].page_content)}")
data[1]

### Wie können wir die Texte in kürzere Abschnitte unterteilen?

Web Demo [Chunk Visualizer](https://huggingface.co/spaces/m-ric/chunk_visualizer)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))
all_splits

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
chroma = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="ibm-granite/granite-embedding-278m-multilingual")
try:
    chroma.delete_collection("documents")
except:
    pass

collection = chroma.get_or_create_collection("documents",embedding_function=sentence_transformer_ef)

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=[item.page_content for item in all_splits], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[item.metadata for item in all_splits], # filter on these!
    ids=[str(id) for id in range(0,len(all_splits))], # unique for each doc
)

In [ ]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Was ist die MMS?"],
    n_results=4,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    #where_document={"$contains":"Oliver"}  # optional filter
)

results

## Aufgaben 

* Trage informationen ein - Findet sie die suche wieder?
* Wie groß sollte die Chuck Size sein?
* Finde die ehemaligen Geschäftsführer


Optional 
* Open AI Embeddings
* BGE Embeddings